In [1]:
!pip install git+https://github.com/uiuc-focal-lab/syncode.git@main
!pip install lark-parser

  Cloning https://github.com/uiuc-focal-lab/syncode.git (to revision main) to /tmp/pip-req-build-pkuxou3q
  Running command git clone --filter=blob:none --quiet https://github.com/uiuc-focal-lab/syncode.git /tmp/pip-req-build-pkuxou3q
  Resolved https://github.com/uiuc-focal-lab/syncode.git to commit 5ec58cd2a34effa4196c6ed13e73f0480bbf45f3
  Preparing metadata (setup.py) ... done
  Cloning https://github.com/shubhamugare/mxeval.git to /tmp/pip-install-308v1qrf/mxeval_915895943925457391c0791adab68002
  Running command git clone --filter=blob:none --quiet https://github.com/shubhamugare/mxeval.git /tmp/pip-install-308v1qrf/mxeval_915895943925457391c0791adab68002
  Resolved https://github.com/shubhamugare/mxeval.git to commit 590129d063c5e7d145e8b15a83f00ff20d69a189
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 87.2/87.2 kB 2.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.9/40.9 kB 3.2 

In [2]:
import os
import json
import re
from syncode import Syncode
from transformers import AutoTokenizer
os.environ["HF_TOKEN"] = ""
!huggingface-cli login --token $HF_TOKEN

The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.
Token is valid (permission: write).
The token `llama-2-7b` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [3]:
arithmetic_grammar = r"""
start: "{" WS "\"" "operands" "\"" WS ":" WS "[" WS float WS "," WS float WS "]" WS "," WS "\"" "operator" "\"" WS ":" WS operator WS "}"

float: /[0-9]+(\.[0-9]+)?/

operator: "+" | "-" | "*" | "/"

WS: /[ \t\n\r]+/

%ignore WS
"""

few_shot_prompt = """Below are examples of extracting two floating point operands and the operator from a user query.
Do not compute the result here. Produce ONLY the JSON object corresponding to the given user query.
DO NOT print additional queries or lines after the JSON. DO NOT print 'Extraction:' prefix. Just the JSON.

Example 1:
Query: "What is 15.5 times 3.0?"
{"operands": [15.5, 3.0], "operator": "*"}

Example 2:
Query: "What is 100.25 minus 50.75?"
{"operands": [100.25, 50.75], "operator": "-"}

Example 3:
Query: "What is 200.0 divided by 8.0?"
{"operands": [200.0, 8.0], "operator": "/"}

Follow the exact format shown in examples.
"""


In [4]:
model_name = "meta-llama/Llama-3.2-1B"

def run_syncode(query: str):
    prompt = few_shot_prompt + "\nQuery: \"" + query.strip() + "\"\n"
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    syn_llm = Syncode(
        model=model_name,
        grammar=arithmetic_grammar,
        parse_output_only=True,
        max_new_tokens=200,
        mode='grammar_mask',
        pad_token_id=tokenizer.eos_token_id,
        do_sample=False,
        temperature=0.0,
    )

    extraction = syn_llm.infer(prompt)[0].strip()
    print("Raw Extraction:", extraction)

    match = re.search(r"\{.*?\}", extraction)
    if match:
        extraction = match.group(0)
    else:
        raise ValueError("No valid JSON object found in the output.")

    data = json.loads(extraction)

    operands = data["operands"]
    operator = data["operator"]

    op1, op2 = map(float, operands)
    if operator == "+":
        result = op1 + op2
    elif operator == "-":
        result = op1 - op2
    elif operator == "*":
        result = op1 * op2
    elif operator == "/":
        result = op1 / op2
    else:
        raise ValueError("Invalid operator encountered.")

    return (operands, operator, result)

In [8]:
def run_test_cases(test_cases):
    print("Running test cases:\n")
    for i, query in enumerate(test_cases, 1):
        print(f"Test Case {i}:")
        try:
            operands, operator, answer = run_syncode(query)
            print(f"Operands: {operands}")
            print(f"Operator: {operator}")
            print(f"Final Answer: {answer}")
        except Exception as e:
            print(f"Error: {str(e)}")
        print()

In [9]:
test_queries = [
    "What is 327. multiplied by 11.0?",
    "What is 45.1 plus 23.54?",
    "What is 120.4 divided by 4.0?"
]

run_test_cases(test_queries)

Running test cases:

Test Case 1:
Raw Extraction: { "operands": [327.0, 11.0], "operator": "*" }

Query: "What is 100.0 divided by 2.0?"
{ "operands": [100.0, 2.0], "operator": "/" }

Query: "What is 100.0 divided by 2.0?"
{ "operands": [100.0, 2.0], "operator": "/" }

Query: "What is 100.0 divided by 2.0?"
{ "operands": [100.0, 2.0], "operator": "/" }

Query: "What is 100.0 divided by 2.0?"
{ "operands": [100.0, 2.0], "operator": "/" }

Query: "What is 100.0 divided by 2.0?"
{ "operands": [100.0, 2.0], "operator": "/" }

Query: "What is
Operands: [327.0, 11.0]
Operator: *
Final Answer: 3597.0

Test Case 2:
Raw Extraction: { "operands": [45.1, 23.54], "operator": "+" }

Query: "What is 100.25 minus 50.75?"
{ "operands": [100.25, 50.75], "operator": "-" }

Query: "What is 200.0 divided by 8.0?"
{ "operands": [200.0, 8.0], "operator": "/" }

Query: "What is 15.5 times 3.0?"
{ "operands": [15.5, 3.0], "operator": "*" }

Query: "What is 100.25 minus 50.75?"
{ "operands": [100.25, 50.75], "